<a href="https://colab.research.google.com/github/peremartra/Rearchitecting-LLMs/blob/main/CH03/CH03_NB01_Model_structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Rearchitecting LLMs
## Surgical Optimization for Hyper-Efficient Models


### Chapter 3: The Internal Structure of Modern Transformers
by [Pere Martra](https://github.com/peremartra)
_____
Colab Environment: GPU T4

Models:
* distilgpt2
* meta-llama/Llama-3.2-1B
* google/gemma-3-270m
* microsoft/Phi-4-mini-instruct
* Llama-3.1-8B-Instruct

_____

In this notebook you'll find the structure of different transformer models.

In [1]:
!pip install -q \
    "transformers==4.51.3" \
    "accelerate==1.3.0" \
    "bitsandbytes"

import torch
import gc
#import psutil
#import os
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, AutoConfig
#from transformers.models.auto.modeling_auto import MODEL_MAPPING
import warnings
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.3 MB/s eta 0:00:00


In [2]:
def clean_memory(model):
    """Limpia memoria"""
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [3]:
def load_model(model_name):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto", #torch.float16,
        device_map="cpu",
        trust_remote_code=True
    )

    return model

# Chapter architectures

##DistilGPT2

We've chosen the DistilGPT2 model for being a base model that represents the essence of a transformer model.

With it we'll explain how the attention layer works and what major blocks form the models.

In [4]:
model = load_model("DistilGPT2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [6]:
config = model.config
clean_memory(model)
print(f"Attention heads: {config.n_head}")
print(f"Head dimensions: {config.n_embd // config.n_head}")

Attention heads: 12
Head dimensions: 64


In [7]:
import inspect
mlp_layer = model.transformer.h[0].mlp
print(inspect.getsource(mlp_layer.forward))

    def forward(self, hidden_states: Optional[Tuple[torch.FloatTensor]]) -> torch.FloatTensor:
        hidden_states = self.c_fc(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.c_proj(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states



In [9]:
init_source_code = inspect.getsource(model.transformer.__init__)
print(init_source_code)

    def __init__(self, config):
        super().__init__(config)

        self.embed_dim = config.hidden_size

        self.wte = nn.Embedding(config.vocab_size, self.embed_dim)
        self.wpe = nn.Embedding(config.max_position_embeddings, self.embed_dim)

        self.drop = nn.Dropout(config.embd_pdrop)
        self.h = nn.ModuleList([GPT2Block(config, layer_idx=i) for i in range(config.num_hidden_layers)])
        self.ln_f = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_epsilon)

        # Model parallel
        self.model_parallel = False
        self.device_map = None
        self.gradient_checkpointing = False
        self._attn_implementation = config._attn_implementation

        # Initialize weights and apply final processing
        self.post_init()



In [11]:
forward_source_code = inspect.getsource(model.transformer.forward)
print(forward_source_code)

    @add_start_docstrings_to_model_forward(GPT2_INPUTS_DOCSTRING)
    @add_code_sample_docstrings(
        checkpoint=_CHECKPOINT_FOR_DOC,
        output_type=BaseModelOutputWithPastAndCrossAttentions,
        config_class=_CONFIG_FOR_DOC,
    )
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool

**DistilGPT2 - Key Insights Mapped to Model Structure:**

• **Classical Stacked Architecture**: 6 identical GPT2Block layers
  → **Line**: `(0-5): 6 x GPT2Block(`

• **Traditional MLP Design**: Simple linear transformation (768→3072→768) with 4x expansion ratio
  → **Lines**:
  ```
  (c_fc): Conv1D(nf=3072, nx=768)
  (c_proj): Conv1D(nf=768, nx=3072)
  ```

• **Multi-Head Attention**: Classic attention implementation using Conv1D layers (c_attn projects to 2304 dimensions for Q, K, V)
  → **Line**: `(c_attn): Conv1D(nf=2304, nx=768)`

• **Pre-Norm Architecture**: LayerNorm applied before attention (ln_1) and MLP (ln_2), not after
  → **Lines**:
  ```
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  ```

• **Compact Embedding Space**: 768-dimensional hidden states with 50,257 vocabulary tokens
  → **Line**: `(wte): Embedding(50257, 768)`

• **Moderate Parameter Count**: ~82M parameters
  → **Pattern**: Calculated from embedding dimensions (50257×768) + layer parameters (visible through 768 dimensions throughout)

• **Classical Width Pruning Target**: The MLP's c_fc layer (768→3072) represents the traditional approach to width pruning
  → **Line**: `(c_fc): Conv1D(nf=3072, nx=768)`

## meta-llama/Llama-3.2-1B

We select Llama-3.2-1B for belonging to the family of models that has popularized the modern structure of Transformers. When analyzing its structure, we'll focus on the two major evolutions compared to the classic architecture: the introduction of MLP blocks with GLU and the optimization of attention with Grouped-Query Attention (GQA).

In [ ]:
model = load_model("meta-llama/Llama-3.2-1B")

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [ ]:
config = model.config
clean_memory(model)
print(f"Attention heads: {config.num_attention_heads}")
print(f"Head dimensions: {config.hidden_size // config.num_attention_heads}")

Attention heads: 32
Head dimensions: 64


**Llama-3.2-1B - Key Insights Mapped to Model Structure:**

• **Modern GLU Architecture**: MLP uses **gating mechanism** (gate_proj + up_proj → down_proj)
  → **Lines**:
  ```
  (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
  (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
  (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
  ```

• **Grouped Query Attention (GQA)**: Q projections use full 2048 dimensions while K/V use only 512
  → **Lines**:
  ```
  (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
  (k_proj): Linear(in_features=2048, out_features=512, bias=False)
  (v_proj): Linear(in_features=2048, out_features=512, bias=False)
  ```

• **Higher Expansion Ratio**: MLP expands from 2048→8192 (4x ratio) but with GLU structure
  → **Lines**:
  ```
  (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
  (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
  ```

• **Bias-Free Design**: No bias parameters throughout - cleaner architecture
  → **Pattern**: `bias=False` appears in all Linear layers throughout the model

• **RMSNorm Normalization**: Uses RMS normalization instead of LayerNorm
  → **Lines**:
  ```
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (norm): LlamaRMSNorm((2048,), eps=1e-05)
  ```

• **Deeper Architecture**: 16 layers vs DistilGPT2's 6 - more **Depth Pruning opportunities**
  → **Line**: `(0-15): 16 x LlamaDecoderLayer(`

• **Rotary Position Embeddings**: Advanced positional encoding that works better for longer sequences
  → **Line**: `(rotary_emb): LlamaRotaryEmbedding()`

• **SiLU Activation**: Smooth activation function in MLP that influences how gating works in the GLU structure
  → **Line**: `(act_fn): SiLU()`

• **Perfect Width Pruning Candidate**: The dual-path MLP structure (gate_proj + up_proj) makes this architecture ideal for demonstrating advanced neuron selection strategies
  → **Lines**:
  ```
  (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
  (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
  ```

## google/gemma-3-270m
Gemma-3 is an evolution of the pattern introduced with Llama, confirming that Llama's pattern is a de facto standard in the industry, adopted by other big players like Google.

Additionally, it allows us to connect with the practical work from Chapter 2. When observing its structure, we'll see that it's almost identical to Llama's, but we'll focus on the subtle implementation variations, like the use of different activation functions.

In [ ]:
model = load_model("google/gemma-3-270m")

In [ ]:
print(model)

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps

In [ ]:
config = model.config
clean_memory(model)
print(f"Attention heads: {config.num_attention_heads}")
print(f"Head dimensions: {config.hidden_size // config.num_attention_heads}")

Attention heads: 4
Head dimensions: 160


**Gemma-3-270M - Key Insights Mapped to Model Structure:**

• **Compact GLU Architecture**: Modern GLU structure (gate_proj + up_proj → down_proj) like Llama but in a much smaller footprint
  → **Lines**:
  ```
  (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
  (up_proj): Linear(in_features=640, out_features=2048, bias=False)
  (down_proj): Linear(in_features=2048, out_features=640, bias=False)
  ```

• **Grouped Query Attention with Normalization**: GQA structure (Q: 640→1024, K/V: 640→256) plus unique **q_norm and k_norm** layers
  → **Lines**:
  ```
  (q_proj): Linear(in_features=640, out_features=1024, bias=False)
  (k_proj): Linear(in_features=640, out_features=256, bias=False)
  (v_proj): Linear(in_features=640, out_features=256, bias=False)
  (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
  (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
  ```

• **Ultra-Lightweight Design**: Only 640 hidden dimensions and 270M parameters
  → **Pattern**: `in_features=640` appears throughout + `Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)`

• **Conservative Expansion Ratio**: 3.2x MLP expansion (640→2048) - more modest than Llama's 4x
  → **Lines**:
  ```
  (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
  (up_proj): Linear(in_features=640, out_features=2048, bias=False)
  ```

• **Excessive Normalization**: Four RMSNorm layers per decoder block - demonstrates modern architecture's emphasis on training stability
  → **Lines**:
  ```
  (input_layernorm): Gemma3RMSNorm((640,), eps=1e-06)
  (post_attention_layernorm): Gemma3RMSNorm((640,), eps=1e-06)
  (pre_feedforward_layernorm): Gemma3RMSNorm((640,), eps=1e-06)
  (post_feedforward_layernorm): Gemma3RMSNorm((640,), eps=1e-06)
  ```

• **Dual Rotary Embeddings**: Both standard and "local" rotary embeddings - advanced positional encoding
  → **Lines**:
  ```
  (rotary_emb): Gemma3RotaryEmbedding()
  (rotary_emb_local): Gemma3RotaryEmbedding()
  ```

• **GELU-Tanh Activation**: Uses PytorchGELUTanh instead of SiLU - shows activation function diversity
  → **Line**: `(act_fn): PytorchGELUTanh()`

• **Scaled Word Embeddings**: `Gemma3TextScaledWordEmbedding` indicates specialized embedding handling
  → **Line**: `(embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)`

• **Perfect Educational Model**: Small enough for laptop experimentation but sophisticated enough to demonstrate all modern techniques - ideal for **Depth Pruning** (18 layers) and **Width Pruning** examples
  → **Line**: `(0-17): 18 x Gemma3DecoderLayer(` + compact dimensions throughout


# Exploring Other Notable Architectures

## microsoft/Phi-4-mini-instruct

We include microsoft/Phi-4-mini-instruct as a case study on diversity in architecture design. With this model, we'll observe two fundamental differences compared to the Llama/Gemma pattern: first, its attention mechanism, which returns to the classic Multi-Head Attention (MHA) implementation; and second, its MLP block, which uses an ingenious fused layer that differs from the three-layer design we've seen.

In [ ]:
model = load_model("microsoft/Phi-4-mini-instruct")

In [ ]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(200064, 3072, padding_idx=199999)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=5120, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_features=20006

In [ ]:
config = model.config
clean_memory(model)
print(f"Attention heads: {config.num_attention_heads}")
print(f"Head dimensions: {config.hidden_size // config.num_attention_heads}")

Attention heads: 24
Head dimensions: 128


**Phi-4-mini-instruct - Key Insights Mapped to Model Structure:**

• **Deep Architecture**: 32 layers, demonstrates **extreme Depth Pruning potential** but requires sophisticated layer selection strategies
  → **Line**: `(0-31): 32 x Phi3DecoderLayer(`

• **Fused GLU Implementation**: `gate_up_proj` combines gating and up projection in a single matrix (3072→16384), then splits internally
  → **Line**: `(gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)`

• **Compact Multi-Head Attention**: Single `qkv_proj` (3072→5120) handles all Q, K, V projections
  → **Line**: `(qkv_proj): Linear(in_features=3072, out_features=5120, bias=False)`

• **Asymmetric Down Projection**: `down_proj` expects 8192 inputs but `gate_up_proj` outputs 16384 - confirms the internal splitting of the fused projection
  → **Lines**:
  ```
  (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
  (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
  ```

• **Residual Dropout Layers**: Explicit dropout for attention and MLP residual connections - additional regularization not seen in Llama
  → **Lines**:
  ```
  (resid_attn_dropout): Dropout(p=0.0, inplace=False)
  (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
  ```

• **Large Vocabulary**: 200K+ tokens vs Llama's 128K - shows how modern models handle larger tokenization schemes
  → **Line**: `(embed_tokens): Embedding(200064, 3072, padding_idx=199999)`

• **Zero Dropout Training**: All dropout probabilities set to 0.0 - indicates this is an inference-optimized checkpoint
  → **Pattern**: `p=0.0` in both `(resid_attn_dropout)` and `(resid_mlp_dropout)` lines


##Llama-3.1-8B-Instruct quantized.

This model has been chosen for two strategic reasons. First, its large size (8B parameters) makes quantization a practical necessity to run it in most environments, allowing us to see how the quantization library replaces its layers. Second, being an -Instruct model, we observe that the architecture is the same as the foundational model and that the ability to follow instructions is added to the model just with fine-tuning.

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [ ]:
model_instruct_quantized = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-3.1-8B-Instruct",
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.float16
    )

In [ ]:
print(model_instruct_quantized)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [ ]:
config = model.config
clean_memory(model)
print(f"Attention heads: {config.num_attention_heads}")
print(f"Head dimensions: {config.hidden_size // config.num_attention_heads}")

Attention heads: 32
Head dimensions: 64


* The `Linear` layers of the Llama-3.2-1B model have been replaced by `Linear4bit` layers.

## Summary

In this notebook we've inspected five different models to understand their internal components.

DistilGPT2 is the one that has the classic structure: simple MLP with 4x expansion and traditional Multi-Head Attention.

Llama-3.2-1B introduced the two key innovations of modern models: GLU architecture in the MLP (gate_proj + up_proj → down_proj) and Grouped Query Attention to optimize memory.

Gemma-3 confirmed that the Llama pattern has become standard, maintaining the same GLU structure but adding additional normalizations and using GELU-Tanh instead of SiLU as activation in the GLU structure.

With Phi-4 we've seen that there's room for variations: its fused GLU implementation (gate_up_proj) is more memory efficient, and its return to classic MHA shows that not all optimizations are universal.

The key lesson is that, regardless of variations, all these models share similar components we can modify: stacked layers for depth pruning, MLP projections for width pruning, and attention mechanisms we can bypass dynamically.

Understanding these structures is fundamental for the optimization techniques we'll apply in the upcoming chapters.